# SnorkelMED - Identifying Opioid-Induced Respiratory Depression  

The purpose of this analysis is to probabilistically identify which patient visits included an opioid-induced respiratory depression (OIRD) event. 

In [ ]:
%matplotlib inline
import os
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

from snorkel.labeling import labeling_function, PandasLFApplier, LFAnalysis, filter_unlabeled_dataframe
from snorkel.labeling.model import MajorityLabelVoter, LabelModel
from snorkel.analysis import get_label_buckets

majority_model = MajorityLabelVoter()
label_model = LabelModel(cardinality=2, verbose=True)

import helper as hlp
import importlib
importlib.reload(hlp)

# global variables
ABSTAIN = -1; CONTROL = 0; CASE = 1

In [ ]:
# load updated training/dev/valid data after labeling from previous round
df_train, df_dev, df_valid, df_test = hlp.load_data(round=3)

# re-attach numeric data to reflect any updated rules
df_train, df_dev = hlp.reattach_numeric_data(df_train, df_dev)

# keep confounding diagnoses visits available
confounding_diagnosis_present = pd.read_csv('../sd_structured/icd/visits_with_confounding_icd_codes.csv')

# made changes in code in to loading confounding diagnoses, so eliminating the redundant columns
df_train.drop(['condition_start_date', 'cva', 'sepsis'], axis = 1, inplace = True)
df_dev.drop(['condition_start_date', 'cva', 'sepsis'], axis = 1, inplace = True)

In [ ]:
# store Y values for ease of evaluation
Y_dev = df_dev['label'].values
Y_dev = np.where(Y_dev=='case', 1, 0) 

#Y_valid = df_valid['label'].values

# Round 3 - Add More Learning Functions

When applying Round 2 Learning Functions, the top probabilities didn’t perform well (only 1 out of 10 were flagged) – notably, very few had any naloxone administration. They were all in the “study group” of “case” based on AHRQ SQL criteria (indicating post-operative respiratory failure), though. So far, there are no manually-adjudicated cases in the dev set (n=70) that do not have a naloxone administration. Therefore, hyper-parameter tuning that strongly weights the naloxone rule would be preferred. Unfortunately, that rule only has ~1.7% coverage in the training set, so additional rules are needed. 

In [ ]:
# create dictionary to keep track of rule names for easier reference later
lfd = dict()

@labeling_function()
def LF_naloxone_admin(x):
    if x['naloxone_admin_prob'] >= 0.75:
        if x['counts_naloxone_effective'] > 0:
            return CASE
        elif x['counts_naloxone_NOT_effective'] > 0:
            return CONTROL
        else:
            return CASE
    elif x['naloxone_admin_prob'] < 0.75:
        return CONTROL
    else:
        # if missing
        return ABSTAIN
lfd['LF_naloxone_admin'] = 0

@labeling_function()
def LF_counts_naloxone(x):
    # in round 3, adding checks for whether naloxone is effective
    # this is similar to the naloxone_admin rule but tries to 
    # capture patients where it was mentioned but not documented as administered
    # per the drug_exposure table
    if x['counts_naloxone'] > 0: 
        if x['counts_naloxone_effective'] > 0:
            return CASE
        elif x['counts_naloxone_NOT_effective'] > 0:
            return CONTROL
        # default to case if not suggestion of working or not
        else:
            return CASE
    return ABSTAIN
lfd['LF_counts_naloxone'] = max(lfd.values()) + 1

"""
@labeling_function()
def LF_respiratory_failure_any(x):
    # this ended up being a heavily-weighted rule to lean controls toward cases
    # but it had excellent coverage originally - a chat on Spectrum suggested that if 
    # you only have a few LFs, highly-specific rules are preferred over high-coverage rules
    if '1' in x['respiratory_failure_any'].lower(): 
        # if there is a confounding diagonsis that would make respiratory failure likely
        if x['visit_occurrence_id'] in confounding_diagnosis_present['visit_occurrence_id'].unique():
            return CONTROL
        # if going on a ventilator, not likely to be related to opioids
        elif 'yes' in x['eligible_vent'].lower():
            return CONTROL
        # or if there is a lack of any other evidence to suggest the patient is a case
        elif x['counts_naloxone'] == 0 and \
             np.isnan(x['naloxone_admin_prob']) and \
             x['counts_altered_mental_status'] == 0 and \
             x['counts_narcotic_overdose'] == 0 and \
             x['counts_hypoxia'] == 0 and \
             x['counts_decrease_opioids'] == 0:
            return CONTROL
        #else:
        #    return CASE
    return ABSTAIN
lfd['LF_respiratory_failure_any'] = max(lfd.values()) + 1
"""

@labeling_function()
def LF_respiratory_failure_any(x):
    # this ended up being a heavily-weighted rule to lean controls toward cases
    # but it had excellent coverage originally - a chat on Spectrum suggested that if 
    # you only have a few LFs, highly-specific rules are preferred over high-coverage rules
    if '1' in x['respiratory_failure_any'].lower(): 
        # if going on a ventilator, not likely to be related to opioids
        if 'yes' in x['eligible_vent'].lower():
            return CONTROL
    return ABSTAIN
lfd['LF_respiratory_failure_any'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_resp_care_notes(x):
    if x['counts_resp_care_notes'] == 0:
        return CONTROL
    return ABSTAIN
lfd['LF_counts_resp_care_notes'] = max(lfd.values()) + 1

In [ ]:
@labeling_function()
def LF_counts_no_acute_events(x):
    # this ended up being a heavily-weighted rule to lean cases toward controls
    # so additional logic checks to see if other "acute" things happened during the visit
    #   (even if they didn't happen for several days where this was recorded)
    if x['counts_no_acute_events'] > 0:
        if x['counts_naloxone'] + x['counts_rapid_response'] + x['counts_altered_mental_status'] + \
            x['counts_narcotic_overdose'] + x['counts_hypoxia'] > 0:
            return ABSTAIN
        return CONTROL
    return ABSTAIN
lfd['LF_counts_no_acute_events'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_altered_mental_status(x):
    if x['counts_altered_mental_status'] > 0:
        # first check to see whether another condition could be responsible
        if x['visit_occurrence_id'] in confounding_diagnosis_present['visit_occurrence_id'].unique(): 
            return CONTROL
        # if no other condition, add support being a case
        return CASE
    return ABSTAIN
lfd['LF_counts_altered_mental_status'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_narcotic_overdose(x):
    if x['counts_narcotic_overdose'] > 0:
        return CASE
    return ABSTAIN
lfd['LF_counts_narcotic_overdose'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_hypoxia(x):
    if x['counts_hypoxia'] > 0:
        return CASE
    return ABSTAIN
lfd['LF_counts_hypoxia'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_decrease_opioids(x):
    if x['counts_decrease_opioids'] > 0:
        return CASE
    return ABSTAIN
lfd['LF_counts_decrease_opioids'] = max(lfd.values()) + 1

@labeling_function()
def LF_confounding_diagnosis_for_rrt(x):
    # added new rules in round #3 (i.e., for heart disease [to include arrhythmias] & 
    # other respiratory disease causes)
    if x['visit_occurrence_id'] in confounding_diagnosis_present['visit_occurrence_id'].unique(): 
        return CONTROL
    elif x['counts_rapid_response'] > 0:
        return CASE
    return ABSTAIN
lfd['LF_confounding_diagnosis_for_rrt'] = max(lfd.values()) + 1

@labeling_function()
def LF_extended_vent_time(x):
    # if on the vent for 4 or more days
    if 'yes;;yes;;yes;;yes' in x['eligible_vent'].lower(): 
        return CONTROL
    return ABSTAIN
lfd['LF_extended_vent_time'] = max(lfd.values()) + 1

In [ ]:
# Round 3 Additions
@labeling_function()
def LF_counts_no_pain_meds(x):
    # low coverage but found in the dev set review last time
    if x['counts_no_pain_meds'] > 0:
        return CONTROL
    return ABSTAIN
lfd['LF_counts_no_pain_meds'] = max(lfd.values()) + 1

@labeling_function()
def LF_no_support(x):
    # find everything that could result in a CASE
    if x['counts_naloxone'] == 0 and \
        np.isnan(x['naloxone_admin_prob']) and \
        x['counts_altered_mental_status'] == 0 and \
        x['counts_narcotic_overdose'] == 0 and \
        x['counts_hypoxia'] == 0 and \
        x['counts_decrease_opioids'] == 0 and \
        x['counts_rapid_response'] == 0 and \
        '1' not in x['respiratory_failure_any'].lower():
        return CONTROL
    return ABSTAIN
lfd['LF_no_support'] = max(lfd.values()) + 1

In [ ]:
# combine all relevant LFs
lfs = [LF_naloxone_admin,
      LF_counts_naloxone,
      LF_respiratory_failure_any,
      LF_counts_resp_care_notes,
      LF_counts_no_acute_events,
      LF_counts_altered_mental_status,
      LF_counts_narcotic_overdose,
      LF_counts_hypoxia,
      LF_counts_decrease_opioids,
      LF_confounding_diagnosis_for_rrt,
      LF_extended_vent_time,
      LF_counts_no_pain_meds,
      LF_no_support,
      ]

# apply LFs
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_dev = applier.apply(df=df_dev)

In [ ]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

In [ ]:
LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=Y_dev)

## Explore Rules

In [ ]:
# do naloxone mentions pick up on any administrations that the admin probability doesn't?
df_dev.iloc[(L_dev[:, lfd['LF_naloxone_admin']]==ABSTAIN) & \
           (L_dev[:, lfd['LF_counts_naloxone']]!=ABSTAIN)]

In [ ]:
df_train.iloc[(L_train[:, lfd['LF_naloxone_admin']]==ABSTAIN) & \
           (L_train[:, lfd['LF_counts_naloxone']]!=ABSTAIN)][:10]

In [ ]:
df_train.iloc[L_train[:, lfd['LF_naloxone_admin']] == CONTROL]['naloxone_admin_prob'].head()

In [ ]:
buckets = get_label_buckets(L_dev[:, lfd['LF_confounding_diagnosis_for_rrt']], 
                            L_dev[:, lfd['LF_counts_no_acute_events']])
df_dev[['label', 'counts_no_acute_events', 'counts_rapid_response', 'cond_sepsis', 'cond_cva',
       'cond_resp_disease', 'cond_cv_disease']] \
    .iloc[buckets[(CONTROL, CONTROL)]]

In [ ]:
# attempting to embed altered mental status within the RRT confounding rule 
# alternatively, add the confounding diagnoses to altered mental status 
df_dev.iloc[(L_dev[:, lfd['LF_confounding_diagnosis_for_rrt']]==CONTROL) & \
           (L_dev[:, lfd['LF_counts_altered_mental_status']]==CASE)]

`eligible_vent` had low coverage & low empirical accuracy, and the introduction of `extended_vent_time` (which has higher empirical accuracy) has helped classification. I'm going to try incorporating `eligible_vent` into `respiratory_failure_any` because that rule heavily influences some of the misclassifications. It's rare to go on a vent after being over-narcatized (particularly if not mentioned elsewhere). To keep the high-coverage rule in place, moving eligible vent to resp failure any. If there's a "yes" in `eligible_vent`, return `respiratory_failure_any` as CONTROL. 

It seems that rules with high coverage have get greater influence on the final classification (at least based on what I'm seeing with `respiratory_failure_any` and `confounding_diagnosis_present`). 

`altered_mental_status` has good coverage but poor empirical accuracy. Now that we have the `confounding_diagnosis_present` rule, I have added those criteria to `altered_mental_status` and it went from about 26% empirical accuracy to 74% empirical accuracy. 

In [ ]:
lfs.remove(LF_counts_altered_mental_status)
lfs.remove(LF_confounding_diagnosis_for_rrt)
lfs

In [ ]:
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_dev = applier.apply(df=df_dev)

## Voting

In [ ]:
# ensure the code can run
label_model.fit(L_train=L_train, Y_dev = Y_dev, 
                n_epochs = 4000, lr = 0.004, #l2 = 0.01,
                optimizer = 'adamax', lr_scheduler = 'step', #prec_init = 0.7,
                log_freq = 100, seed = 987)
analysis = LFAnalysis(L=L_train, lfs=lfs).lf_summary(est_weights=label_model.get_weights())

### Hyper-Parameter Tuning

In [ ]:
# create empty dataframe to hold learned weights for each rule
df_cols = ['n_epochs', 'lr', 'lr_scheduler']
df_cols.extend(analysis.index)

# specify potential hyperparameters
n_epochs = [2000, 4000]
lr = [0.001, 0.005, 0.01]
lr_scheduler = ['step', 'exponential', 'linear']

# tune - in round 2, started adding new seeds also (e.g., 987, 456, & 123)
SEED = 123
df_tune, df_tune_long = hlp.label_model_tuning(lfs, df_cols, 
                                               L_train, L_dev, Y_dev, 
                                               n_epochs, lr, lr_scheduler,
                                               seed = SEED)

In [ ]:
# review best accuracies among development set
df_tune[df_tune['accuracy'] == np.max(df_tune['accuracy'])]

In [ ]:
for scheduler in lr_scheduler:
    g = sns.FacetGrid(df_tune_long[df_tune_long['lr_scheduler']==scheduler], 
                      col='lr', hue='learning_function', col_wrap=3, height=4)
    g = (g.map(plt.scatter, 'n_epochs', 'learned_weight')
            .add_legend()
            .fig.suptitle('Learned Weights Using ' + str(scheduler) + ' Scheduler', 
                          y=1.05, fontsize=16))

In [ ]:
label_model.fit(L_train=L_train, Y_dev = Y_dev, n_epochs = 2000, lr = 0.001, optimizer = 'adamax', 
                lr_scheduler = 'exponential', log_freq = 100, seed = SEED)

In [ ]:
LFAnalysis(L=L_train, lfs=lfs) \
    .lf_summary(est_weights = label_model.get_weights()) \
    .sort_values(by='Learned Weight', ascending=False)

In [ ]:
majority_acc = majority_model.score(L=L_dev, Y=Y_dev)["accuracy"]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_dev, Y=Y_dev)["accuracy"]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

In [ ]:
# assign probabilities from either majority vote or label model
#gen_probs_train = majority_model.predict_proba(L=L_train)
#gen_probs_dev = majority_model.predict_proba(L=L_dev)

gen_probs_train = label_model.predict_proba(L=L_train)
gen_probs_dev = label_model.predict_proba(L=L_dev)

In [ ]:
hlp.plot_probabilities_histogram(gen_probs_train[:, CASE])

In [ ]:
hlp.plot_probabilities_histogram(gen_probs_dev[:, CASE])

In [ ]:
# How many unlabeled rows? 
df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=df_train, y=gen_probs_train, L=L_train
)
print(str(df_train.shape[0] - df_train_filtered.shape[0]) + ' unlabeled rows in training set.')

## Review Assigned Probabilities

In [ ]:
dev_with_probs = df_dev.copy()
dev_with_probs['label_model_prob'] = gen_probs_dev[:, CASE]

cases_low_prob = dev_with_probs[(dev_with_probs['label']=='case') & \
                                (dev_with_probs['label_model_prob']<=0.5)]
controls_high_prob = dev_with_probs[(dev_with_probs['label']=='control') & \
                                    (dev_with_probs['label_model_prob']>0.5)]

cols_for_review = ['label',  'age_on_admission',
     'label_model_prob', 'naloxone_admin_prob', 'eligible_vent', 'respiratory_failure_any',
     'counts_naloxone', 'counts_resp_care_notes', 'counts_rapid_response', 
     'counts_no_acute_events', 'counts_altered_mental_status', 'counts_narcotic_overdose',
     'counts_hypoxia','counts_decrease_opioids', 'counts_naloxone_effective', 'counts_naloxone_NOT_effective',
     'cond_sepsis', 'cond_cva', 'cond_resp_disease', 'cond_cv_disease']

cases_low_prob[cols_for_review]

In [ ]:
# which rules were flagging?
L_dev[cases_low_prob.index.values]

In [ ]:
controls_high_prob[cols_for_review]

In [ ]:
L_dev[controls_high_prob.index.values]

In [ ]:
lfs

In [ ]:
# exploration of associations 
from matplotlib  import cm

df_fig = dev_with_probs.copy()
colors = ['red' if p=='case' else 'blue' for p in df_fig['label']]
plt.scatter(df_fig['naloxone_admin_prob'],
            df_fig['label_model_prob'],
            c=colors)

In [ ]:
# assigned probabilities of cases
plt.hist(dev_with_probs[dev_with_probs['label']=='case']['label_model_prob'], bins=20)

In [ ]:
# assigned probabilities of controls
plt.hist(dev_with_probs[dev_with_probs['label']=='control']['label_model_prob'], bins=20)

In the past round of reviews, a lot of patients were labeled as **cases** even though they didn't have an naloxone administration. Out of curiosity, how well are the updated rules labeling patients as cases who have a naloxone administration?

In [ ]:
# attach LabelModel predictions to dataframes
train_with_probs = df_train.copy()
train_with_probs['label_model_prob'] = gen_probs_train[:, 1]

In [ ]:
# probabilities with any naloxone admin
plt.hist(train_with_probs[train_with_probs['naloxone_admin_prob']>=0.75]['label_model_prob'], bins=10)

In [ ]:
# probabilities with CASE on naloxone_admin LF
plt.hist(train_with_probs.iloc[L_train[:, lfd['LF_naloxone_admin']] == CASE]['label_model_prob'], bins=10)

In [ ]:
# probabilities with low confidence of actual naloxone admin
plt.hist(train_with_probs[train_with_probs['naloxone_admin_prob']<0.75]['label_model_prob'], bins=10)

In [ ]:
# probabilities with NO naloxone admin
plt.hist(train_with_probs[train_with_probs['naloxone_admin_prob'].isnull()]['label_model_prob'], bins=10)

In [ ]:
# attach LabelModel predictions to dataframes
train_with_probs = df_train.copy()
train_with_probs['label_model_prob'] = gen_probs_train[:, 1]

In [ ]:
# extract top 20 highest probabilities from train set
top_probs = train_with_probs.nlargest(n=20, columns='label_model_prob')

# send half to the dev set & half to the valid set
visits_for_dev = top_probs['visit_occurrence_id'].sample(frac=0.5, random_state=123)
visits_for_valid = top_probs[~np.isin(top_probs['visit_occurrence_id'], visits_for_dev)]['visit_occurrence_id']

# concatenate the respective training set rows to dev & valid sets
df_dev4 = pd.concat([df_dev, df_train[df_train['visit_occurrence_id'].isin(visits_for_dev)]], sort=True)
df_valid4 = pd.concat([df_valid, df_train[np.isin(df_train['visit_occurrence_id'], visits_for_valid)]], sort=True)

# remove the rows from the training set
df_train4 = df_train.drop(top_probs.index)

assert df_dev4.shape[0] == df_dev.shape[0] + 0.5*top_probs.shape[0]
assert df_valid4.shape[0] == df_valid.shape[0] + 0.5*top_probs.shape[0]
assert df_train4.shape[0] == df_train.shape[0] - top_probs.shape[0]
assert not np.isin(df_train4['visit_occurrence_id'], df_dev4['visit_occurrence_id']).any()
assert not np.isin(df_train4['visit_occurrence_id'], df_valid4['visit_occurrence_id']).any()

In [ ]:
# export for manual review - making a copy for being labeled/manipulated & 1 without
#df_train4.to_csv('./train_set4.csv', index=False)
#df_train4.to_csv('./train_set4_labeled.csv', index=False)
#df_dev4.to_csv('./dev_set4.csv', index=False)
#df_dev4.to_csv('./dev_set4_labeled.csv', index=False)
#df_valid4.to_csv('./valid_set4.csv', index=False)
#df_valid4.to_csv('./valid_set4_labeled.csv', index=False)

In [ ]:
# export working file for building code - not to be used for final analysis
train_with_probs.to_csv('./train_set_final.csv', index=False)